In [2]:
# 猫眼电影介绍url
# http://maoyan.com/films/1217236

import requests,time,random
# from fake_useragent import UserAgent
import json,csv,os
import pandas as pd

USER_AGENTS = [ "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)", "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)", "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)", "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)", "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)", "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)", "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)", "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)", "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6", "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1", "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0", "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5", "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6", "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20", "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52", ]

class Spidermaoyan():
	headers = {
		    "User-Agent": random.choice(USER_AGENTS),
		    "Host":"m.maoyan.com",
		    "Referer":"http://m.maoyan.com/movie/1217236/comments?_v_=yes"
		}
	
	def __init__(self,url,time):
		self.url = url
		self.time = time
		
	# 发送get请求
	def get_json(self):
		# 发送get请求
		response_comment = requests.get(self.url, headers=self.headers)
		json_comment = response_comment.text
		json_comment = json.loads(json_comment)
		# print(json_comment)
		return json_comment
	
	# 获取数据并存储
	def get_data(self,json_comment):
		json_response = json_comment["cmts"]  # 列表
		print(len(json_response))
		list_info = []
		for data in json_response:
			cityName = data["cityName"]
			content = data["content"]
			if "gender" in data:
				gender = data["gender"]
			else:
				gender = 0
			nickName = data["nickName"]
			userLevel = data["userLevel"]
			score = data["score"]
			list_one = [self.time,nickName,gender,cityName,userLevel,score,content]
			list_info.append(list_one)
		print(list_info)
		self.file_do(list_info)
	
	# 存储文件
	def file_do(self,list_info):
		# 获取文件大小
		file_size = os.path.getsize(r'C:\Users\Administrator\Desktop\maoyan.csv')
		if file_size == 0:
			# 表头
			name = ['评论日期', '评论者昵称', '性别', '所在城市','猫眼等级','评分','评论内容']
			# 建立DataFrame对象
			file_test = pd.DataFrame(columns=name, data=list_info)
			# 数据写入
			file_test.to_csv(r'C:\Users\Administrator\Desktop\maoyan.csv', encoding='utf_8_sig', index=False)
		else:
			with open(r'C:\Users\Administrator\Desktop\maoyan.csv', 'a+',encoding='utf_8_sig', newline='') as file_test:
				# 追加到文件后面
				writer = csv.writer(file_test)
				# 写入文件
				writer.writerows(list_info)

def spider_maoyan():
	# 猫眼电影短评接口
	offset = 0
	# 电影是2018.9.21上映的
	startTime = '2018-09-21'
	day = [22,23,24,25,26,27,28,29,30,1,2,3,4,5,6]
	j = 0
	page_num = int(20000/15)
	for i in range(page_num):
		comment_api = 'http://m.maoyan.com/mmdb/comments/movie/1217236.json?_v_=yes&offset={0}&startTime={1}%2021%3A09%3A31'.format(offset,startTime)
		s0 = Spidermaoyan(comment_api,startTime)
		json_comment = s0.get_json()
		if json_comment["total"] == 0: # 当前时间内评论爬取完成
			if j < 9:  # 九月份(9天)
				startTime = '2018-09-%d'%day[j]
			elif j >= 9 and j < 15:  # 十月份(6天)
				startTime = '2018-10-%d'%day[j]
			else: # 全部爬完
				break
			offset = 0
			j = j + 1
			continue
		s0.get_data(json_comment)
		offset = offset + 15
		# time.sleep(1)
	

if __name__ == '__main__':
	spider_maoyan()

15
[['2018-09-21', '点点', 2, '盱眙', 4, 4.5, '校园暴力真的是一个值得深思的问题，因为它到现在依然没有得到解决。\n原著党表示，改编还是不错的，毕竟要过审，有些敏感话题还是要回避的，但是已经挺好了，算是回忆了下青春~'], ['2018-09-21', '像', 2, '重庆', 2, 5, '很好看，影片特现实，确实很好看'], ['2018-09-21', 'qEi855206446', 0, '嘉兴', 2, 5, '悲伤逆流成河是有关校园欺凌题材，校园欺凌，听起来还是很熟悉的，毕竟谁的学生时代没经历过点冷眼嘲讽和孤立呢。整部电影给我更多的感觉是压抑，流言的可怕真的是想象不到，剧中句句扎心让人深思……'], ['2018-09-21', 'qTc369881142', 1, '孝昌县', 2, 5, '比较贴切real'], ['2018-09-21', '喜来婚礼💗茄琪', 0, '广元', 1, 5, '很有教育意义，'], ['2018-09-21', 'A清心丸zi', 0, '中山', 3, 5, '挺好看的，真实不庸俗，很感人，希望现实的社会多一些温暖的阳光吧！'], ['2018-09-21', 'rj1981888', 2, '锦州', 4, 4, '齐铭，人如其名，只是想找一个可以和自己身份地位相当的女孩。他对易遥的好大概可以看作是一种怜惜，但却不是爱情。相比较而言，顾森西爱的更加发自内心，也更加真实。他不仅爱易遥的外表，也爱她的灵魂。为了他可以奋不顾身，连命都不要。这才是真正的爱情。爱情不是要说的，而是要经受住考验。'], ['2018-09-21', '草莓味彬彬彬', 2, '牟平', 2, 5, '两个字 看哭'], ['2018-09-21', 'FX19980701', 0, '武汉', 2, 4, '女孩子就是应该温柔中带点暴躁！因为毕竟身边没有一个“顾森西”'], ['2018-09-21', 'oBR963070866', 2, '遵义', 2, 5, '电影很棒、各方面'], ['2018-09-21', 'F泠然', 0, '烟台', 2, 5, '落落导演的电影果然没让我失望。\n这是我所看过的最疼痛的电影，每一个镜头和台词都很痛。希望世界上永远没有校园暴力。'], ['2018-

15
[['2018-09-21', 'Zy', 2, '洛阳', 4, 5, '很真实。很感人。'], ['2018-09-21', 'FUN_LOVE_JIA', 1, '北京', 3, 4.5, '悲伤，逆流成河。'], ['2018-09-21', '莫回头', 0, '南川', 4, 5, '反映这个社会，希望引起这个社会的反思'], ['2018-09-21', '等无此人', 1, '珠海', 2, 5, '很值得去看，校园欺凌真的是太恐怖了，希望所有人的青春，都可以被温柔对待'], ['2018-09-21', '用青春换浪漫', 0, '广州', 2, 5, '特别好看，也给我哭够呛，大家去了记得带纸，哭的痛快'], ['2018-09-21', 'nan々', 2, '湖州', 3, 5, '有点压抑 还不错啦'], ['2018-09-21', 'AA🐯绅士-虎儿', 0, '大连', 3, 4.5, '演的不错，女一号，2'], ['2018-09-21', 'Yun💓', 0, '北京', 5, 5, '非常好，久久不能平静\n'], ['2018-09-21', '张怡凡_baby', 2, '襄阳', 3, 2, '不符合逻辑 老师不会做心脉复苏这是瞎搞啊'], ['2018-09-21', '🚶', 0, '亳州', 1, 5, '很有感触！！！'], ['2018-09-21', 'Zzb618426956', 0, '青岛', 3, 0.5, '烂片，当个小说还行，拍成电影还不足，没有任何原因，很生硬的拍成这个样'], ['2018-09-21', 'upG418508951', 0, '常州', 2, 5, '中学时代就已经在看这本书。虽然改编了，变成了教育主题的，算回忆吧。原著很虐！'], ['2018-09-21', '呼啦co', 0, '兰州', 3, 3, '恒大影院，先是自助取票机取不了票，然后看电影看到一半多大屏幕黑掉，真是醉人，等了那么久还修不好，完了还让我们换场次，态度也很冷淡，活久见，WTF？'], ['2018-09-21', 'MCP86592386', 0, '西安', 2, 5, '电影挺好看的'], ['2018-09-21', '小郁同学', 0, '扬州', 3, 4.5, '看了《悲伤逆流成河

15
[['2018-09-21', 'ReJ291924291', 2, '成都', 2, 0.5, '评分是逗我的吧，难看到要死。'], ['2018-09-21', 'xsqmmd329', 2, '朝阳', 1, 5, '悲伤逆流成河'], ['2018-09-21', '三毛61', 0, '呼和浩特', 2, 5, '好看！！！！'], ['2018-09-21', 'A101_', 0, '江阴', 3, 5, '很现实 很好看 超级满意'], ['2018-09-21', 'aoR453619887', 0, '上海', 3, 5, '泪目！！！！！'], ['2018-09-21', '小云云🍃', 2, '常德', 3, 5, '致我逝去的青春'], ['2018-09-21', 'A.宁', 1, '绥中', 2, 5, '电影不错，我还想在看一遍。'], ['2018-09-21', 'Chiyyu', 2, '哈尔滨', 1, 5, '很真实，看到心坎里去了。让很多人想到过去，过去的那个自己。有些人带着恨，有些人带着愧疚。但愿世界多一份美好和温暖。'], ['2018-09-21', '哈特切克张', 2, '重庆', 1, 4.5, '哭惨了。。。'], ['2018-09-21', '鹿鹿子晗', 0, '抚州', 2, 5, '好看，最后眼泪都出来了'], ['2018-09-21', '灰色糯米团', 0, '南京', 3, 0, '真是超级无敌大烂片！！！不论是女主，男主，女配，男配，都是各种作死！女主一开始和妈妈说就不会发生那么多事了好吗！顾森湘出事，和弟弟有那么大的关系，为什么和女主隔阂。还有男配！和女主青梅竹马那么多年，比不过一个一见钟情的人，醉醉的！还有！最最最重要的，学校的老师呢！老师呢！老师呢！摆设还是标本，出事时候一个老师都没有！事后主任打什么马后炮，事情都不调查还**逼！这都是什么鬼！女主说什么，不要对我好，会觉得理所当然，会践踏心什么的，都是什么垃圾话。乱七八糟，不知道这个电影讲的什么鬼！'], ['2018-09-21', '粉墨97', 0, '瑞昌', 2, 4, '初中的时候看过小说所以这次出了电影就来看了 感觉还可以吧  亲情爱情都演绎的不错'], ['2018-09-21', 'ZEq62467244

15
[['2018-09-21', 'Msunshineg', 2, '无锡', 2, 5, '看影片的时候，特别有体会，我哥哥当时上学的时候就受班里人欺负，我知道后直接找到那个**，后来那帮人收敛了，这种校园暴力应该受关注，影片引发的问题可谓校园中无处不在……给10分，拍的很好，总觉得影片时间有点短……'], ['2018-09-21', '儒雅的爱自己8', 0, '新郑', 1, 3, '那个，需要知道的是还不错，让人们关注了校园霸凌事件。'], ['2018-09-21', 'Jmr678914614', 0, '连云港', 2, 5, '男一男二都帅，女二太好看'], ['2018-09-21', '+c', 2, '南阳', 3, 5, '拒绝校园欺凌 希望可以换位思考 有很多的包容 特别喜欢男二'], ['2018-09-21', '_weixin206351665', 0, '武汉', 1, 4, '笑点都在顾森西的身上了。喜欢'], ['2018-09-21', 'zET512367780', 0, '成都', 2, 5, '和原著出入有点大，但有温馨也有悲情，很不错。'], ['2018-09-21', 'Zrf610694718', 0, '中山', 3, 5, '吵闹地进去 安静地离开\n悲伤逆流成河\n请给女主一丝温暖吧 不要再伤害她了\n这一年看过最好看的一出，哪怕是我不是药神，全程我也打了两次哈欠，唯有这部戏让我从头到尾走进画面里，真的很心疼女主。\n看完后彻彻底底记住了女主任敏，以后都会粉你拍的电影。\n影片从头到尾，有阴暗的一幕，亦有温暖的画面，印象最深刻的是女主的妈妈牵着女主说，妈带你去看病。女主彻底泪崩，在即将陷入崩溃边缘，母亲紧紧握着她的手，如救命稻草一般让易遥看到阴暗的世界里一处微弱的光。男主的角色让我又爱又恨，爱的是他给予了女主种种关心和帮助，可恨的是他始终不懂易遥呀。最让我觉得温暖的角色是顾森西，可惜字数有限还有很多想写。\n极力推荐大家去看'], ['2018-09-21', 'Mbc261716625', 0, '上海', 4, 5, '很不错的一部'], ['2018-09-21', '满目时光', 0, '上海', 1, 5, '真的也太好看了吧'], ['2018-09-21', '皓霖724', 2, '抚

15
[['2018-09-21', '萌萌凯', 1, '大庆', 5, 5, '还行！！！！！！！！！'], ['2018-09-21', '咚咚爱吃大米饭', 0, '锦州', 3, 3, '总的来说挺好的吧 就是前半段真的矫情死我了 如果是我 我要一个个删耳光 后面的爆发和跳海也是让人唏嘘.........哎..........再吐槽一下 那个某某湘 都走到那么破的地方了 不转头就走 现在这个社会 怎么还敢一个人跑到那种地方！遇到霸凌这种事 还是要告诉大人 告诉老师 告诉警察叔叔！！联合身边所有的大人！！我就不信了！！！'], ['2018-09-21', '流年易碎，封尘', 0, '常州', 2, 5, '很好看，哭死了，就是结局被改了，不过不改成正能量的，原著结局估计也过不了审核，可惜了'], ['2018-09-21', 'ByunAili', 2, '临沂', 1, 5, '真的非常好看，易瑶跳河之前那段真的非常的棒'], ['2018-09-21', 'Kris！！！！！', 0, '齐齐哈尔', 2, 5, '太好了，这片'], ['2018-09-21', '＿xxxx4＿', 2, '合肥', 3, 5, '真的不要太好看，深有感受，演的很到位，还会在来刷一遍！'], ['2018-09-21', 'xTO311170481', 0, '兰州', 2, 5, '太好看了 高中一直看的小说'], ['2018-09-21', 'Leisten_hhoh', 0, '杭州', 4, 5, '我觉得剧情还可以，女主的演技还可以的，而且亲学姐朱丹妮演的也很好，唐小米的坏演的淋漓尽致哈哈哈，被顾森湘小姐姐吸粉，全集泪点就在她离世的时候…超级心疼哎哟…'], ['2018-09-21', '快乐每一天💋', 2, '北京', 4, 5, '顾森西给易瑶的温暖比齐铭多，还是更喜欢顾森西'], ['2018-09-21', '【凄·月】', 0, '沧州', 5, 5, '为数不多的好片子'], ['2018-09-21', '，，，，，，，，，', 0, '增城', 4, 3.5, '应该是该关注学校暴力了，给学生一个良好的环境'], ['2018-09-21', '桔几。', 0, '温岭', 2, 5, '哭死了 看郭敬明写的都哭 虽然和原著有出路 

...

15
[['2018-09-27', '小小小关关啊', 2, '东营', 1, 4.5, '还好吧，看哭了，很现实，很心酸，很无奈，结局是好的'], ['2018-09-27', 'tsxtsxtsx', 0, '呼和浩特', 1, 4.5, '愿所有的青春都被温柔以待'], ['2018-09-27', 'CFI634167802', 0, '南昌', 2, 5, '一般吧，没有特别'], ['2018-09-27', 'XHJ236326', 1, '深圳', 3, 4, '还好吧，不过还是比较喜欢小说'], ['2018-09-27', 'btU458303646', 0, '黄骅', 2, 4.5, '揭示了社会上的一些现象，挺不错的！'], ['2018-09-27', 'teY560906950', 2, '西安', 3, 4, '过的再难，也得坚持下去。'], ['2018-09-27', 'ETF298302402', 0, '龙口', 3, 5, '好看，真好看'], ['2018-09-27', 'BBBB_文', 1, '黄石', 3, 4.5, '我叫顾森西，西是太阳从西边出来的西'], ['2018-09-27', 'Mew889583801', 0, '哈尔滨', 2, 4, '看的得带好纸巾，很感人'], ['2018-09-27', 'yanzi2450', 0, '邯郸', 2, 5, '家庭生活不易，再加上的了那种病，被同学欺凌，看的我泪流不止'], ['2018-09-27', 'FWp15202939139', 2, '西安', 2, 5, '想再看一遍的电影，恰到好处的悲桑。多希望自己的生活中也有一个顾森西，看得懂她，听得懂她，像超人一样帮助着她内心强大，知道最后都不放弃去救她。我不希望顾森湘就那样结束，但也许，那是比起坐上轮椅更好一点的结局吧。'], ['2018-09-27', 'JK九藜然', 0, '合肥', 2, 4, '(๑•́ωก̀๑)还是很虐\n但是比书要安慰些   书更虐'], ['2018-09-27', '居居', 0, '黄石', 2, 5, '看哭了，哈哈哈'], ['2018-09-27', '丶瀞尛妞°980', 2, '唐河', 2, 4.5, '还可以得得得'], ['2018-09-27

15
[['2018-09-28', '鬼姬泡芙', 0, '金华', 5, 5, '真的超级好看，很现实，看哭两次，剧情感人，感染力极强 推荐去看👍👍👍'], ['2018-09-28', 'rxg599321605', 1, '贵阳', 3, 0.5, '我不知道怎么评论？我看的一点感觉都没有就想睡觉？要不是我家说好看我真的不想去？'], ['2018-09-28', '蓉蓉阿\ue022', 0, '芜湖', 3, 5, '好看，看的我都气死了！！！'], ['2018-09-28', '.', 0, '石家庄', 1, 5, '我没去看！！！！'], ['2018-09-28', '王冰', 2, '武汉', 1, 3.5, '有一些点还是蛮感人的。'], ['2018-09-28', 'hdys魅影', 0, '沈阳', 3, 5, '十分，能坑一个是一个😂😂😂'], ['2018-09-28', '爱鑫觉罗郡主', 2, '襄阳', 3, 4, '画面唯美，只是演员的演技还需要放开一些'], ['2018-09-28', '恩雅主义', 0, '昆明', 2, 3.5, '还行吧，没看过小说'], ['2018-09-28', 'PTP734621045', 1, '南京', 4, 5, '不错的片子，值得一看'], ['2018-09-28', '3329猫', 0, '玉溪', 2, 4.5, '一部不得不看的电影，催人泪下？'], ['2018-09-28', '小刘', 1, '天津', 3, 5, '很不错。还想看'], ['2018-09-28', '赵性性。', 2, '郑州', 2, 5, '👍太好看了，感触很深'], ['2018-09-28', '陈小姣88', 2, '岳阳', 2, 5, '女主演技不错，哭戏演的好，够悲惨的了，高中的校园暴力应该比较少吧，多数校园暴力都是在初中'], ['2018-09-28', '薰衣草', 2, '清远', 2, 5, '看影片很沉重，看着可怜的孩子好心疼  幸亏有两个男同学偶尔帮忙  要不可怎么活下去呢'], ['2018-09-28', 'ZC', 0, '呼伦贝尔', 2, 5, '真的特别好看，哭的很痛快，也很减压了']]
15
[['2018-09-28', 'gGA23066921

15
[['2018-09-28', '趁我还念', 2, '西宁', 3, 4, '改编过的，没有小说那么虐'], ['2018-09-28', '工藤新一的毛利兰', 0, '重庆', 4, 5, '话不多说    请带纸巾'], ['2018-09-28', '眼里心里只有你7', 0, '长沙', 1, 4, '个别场景挺有感触，剧情有一点点俗'], ['2018-09-28', '往事如烟long', 0, '郑州', 2, 5, '挺好的，跟女朋友一起看的'], ['2018-09-28', '失眠先生1983', 1, '武汉', 2, 5, '太残酷太黑暗了，想起了我的高中时代，很想抽唐小米几个耳光'], ['2018-09-28', '折曦曦曦', 2, '宜昌', 3, 4.5, '拍的挺好的 结局也改了 挺好的 女主角演的挺好 坏人也绳之以法了'], ['2018-09-28', '倪正泰', 2, '长春', 3, 0.5, '片子看完好扎心！又生气又想暴力！这题材，也真是没谁了！更可气的，破商场的影院，装修前可以凭票根停车，装个修之后，电影票都不好使了！就纳儿闷了，没有电影院给你充人气儿，你家商场还咋立足，别赔光了就好！'], ['2018-09-28', '仙家驾到', 0, '重庆', 3, 5, '给个最高评价  看完就来买原著了，哈哈哈'], ['2018-09-28', 'dxxddd', 0, '西宁', 4, 3.5, '嗯，就，我可能过了那个感动的年纪，不过最后那个提倡加强对校园暴力的监管和对受害者的保护这个点挺好的！'], ['2018-09-28', '雨南', 0, '西安', 2, 5, '超好看的 喜欢女主   '], ['2018-09-28', 'lBA173730739', 0, '芜湖', 3, 5, '校园暴力。。。。'], ['2018-09-28', 'I-Myue🍩', 2, '长春', 2, 5, '超好看，超值得看'], ['2018-09-28', 'Jay木', 0, '衡阳', 2, 5, '超好看哈哈哈'], ['2018-09-28', 'Whisky999', 2, '太谷', 2, 3, '没得意思呀嘿'], ['2018-09-28', 'WFu347458692', 0, '张

15
[['2018-09-28', '阳光sunshine', 2, '北京', 2, 5, '原本以为悲伤是个什么来着。看了电影🎬还是没有让我感到悲伤。或许时间使我已经忘记世界上竟有悲伤这回事。南阿弥陀佛。🙏'], ['2018-09-28', '啦啦啦！', 2, '大连', 3, 4.5, '顾森西是易遥生命里唯一的光'], ['2018-09-28', 'hsF469782003', 1, '安顺', 3, 5, '很不错，不负期望'], ['2018-09-28', 'A.帆总', 0, '深圳', 4, 5, '也许有人会说剪辑太乱 也许有人说负能量太多 也许有人说世界上还是有很多温暖的 也许有人说垃圾片子 可我想给十分 如果可以的话我想给100分 作为初高中被那些“无辜”“路人”的明里暗里的嘲笑和孤立 那种你一进去宿舍就全场安静 你不知道你做错了什么 无影随行的痛苦伴随我直到毕业两三年以后每晚还会在噩梦中频繁出现 每一个参与进去的看热闹的人 你们都是杀人凶手 你们都是恶魔 电影太现实了没有经历过的人很现实的告诉你 社会就是这样的 经历过孤立的我感同身受到满分 虽然和原著有很大出入 但是这个版本少了一些更黑暗的东西 用更短的时间表达出了原文的意思 天堂客满 地狱打烊 耶稣就把我扔在人间。'], ['2018-09-28', 'HG婷', 2, '阜新', 2, 5, '不错。一部反应了校园暴力的好电影'], ['2018-09-28', '椰丝斯威夫特', 0, '乌鲁木齐', 2, 5, '挺好看的不错'], ['2018-09-28', 'zjlคิดถึง', 0, '临汾', 2, 5, '挺好，很现实的一部电影，揭示了现在人性的丑陋，值得一看'], ['2018-09-28', '寄几革', 1, '鞍山', 3, 5, '特别好  小演员们的演技都是杠杠的'], ['2018-09-28', 'mzyooo', 2, '通化', 2, 3.5, '还不错 没看过书 但是觉得很好看 女二号好好看啊 邬君梅的演技真的好 看哭了的一段'], ['2018-09-28', '那你牛逼呀', 0, '邯郸', 3, 5, '很好看很伤感'], ['2018-09-28', '感叹号909', 0, '鞍山', 3, 4.5, '好看吗？快写下你的评论

KeyboardInterrupt: 